In [2]:
from dotenv import load_dotenv
load_dotenv()

True

# Roteamento

In [12]:
from langchain_core.prompts import ChatPromptTemplate as cpt
from langchain_openai.chat_models import ChatOpenAI

model = ChatOpenAI(model= 'gpt-4o-mini')
prompt = cpt.from_template('''Você é um professor de matematica de ensino fundamental capaz de dar respostas muito detalhadas e didáticas. Responda a seguinte pergunta de um aluno: 
Pergunta: {pergunta}''')
chain_matematica = prompt | model 

prompt = cpt.from_template('''Você é um professor de física de ensino fundamental capaz de dar respostas muito detalhadas e didáticas. Responda a seguinte pergunta de um aluno:
Pergunta: {pergunta}''')
chain_fisica = prompt | model

prompt = cpt.from_template('''Você é um professor de história de ensino fundamental capaz de dar respostas muito detalhadas e didáticas. Responda a seguinte pergunta de um aluno:
Pergunta: {pergunta}''')
chain_historia = prompt | model

prompt = cpt.from_template('''{pergunta}''')
chain_generica = prompt | model


In [13]:
from pydantic import BaseModel, Field

prompt = cpt.from_template('Você deve categorizar a seguinte pergunta: {pergunta}')

class Categorizador(BaseModel):
    '''Categoriza as perguntas de alunos do Ensino Fundamental'''
    area_conhecimento: str = Field(description='A área de conhecimento da pergunta feita pelo aluno. \
    Deve ser "física", "matemática" ou "história". Caso não se enquadre em nenhuma delas, retorne "outra"')

model_estruturado = prompt | model.with_structured_output(Categorizador)
model_estruturado.invoke('Quanto é 1+1?')

Categorizador(area_conhecimento='matemática')

### Criando estrutura de roteamento

In [14]:
from langchain_core.runnables import RunnablePassthrough

chain = RunnablePassthrough().assign(categoria=model_estruturado)
chain.invoke({'pergunta': 'Quando foi a independencia do Brasil?'})

{'pergunta': 'Quando foi a independencia do Brasil?',
 'categoria': Categorizador(area_conhecimento='história')}

In [15]:
def route(input):
    if input['categoria'].area_conhecimento == 'matemática':
        return chain_matematica
    elif input['categoria'].area_conhecimento == 'física':
        return chain_fisica
    elif input['categoria'].area_conhecimento == 'história':
        return chain_historia
    return chain_generica

In [18]:
chain = RunnablePassthrough().assign(categoria=model_estruturado) | route
resposta = chain.invoke({'pergunta': 'Por que a fatoração de 0 é 1?'})
print(resposta.content)

Essa é uma pergunta muito interessante e uma ótima oportunidade para entender um conceito importante na matemática! Vamos esclarecer a questão da fatoração de 0.

Primeiro, precisamos entender o que significa "fatoração". Fatoração é o processo de escrever um número como o produto de outros números, chamados de fatores. Por exemplo, a fatoração de 6 é 2 × 3, porque 2 e 3 são fatores de 6.

Agora, quando falamos de "0" e "fatoração", na verdade, estamos nos referindo a uma notação especial chamada "fatorial". O fatorial de um número \( n \) (escrito como \( n! \)) é o produto de todos os números inteiros positivos até \( n \). Por exemplo:

- \( 3! = 3 \times 2 \times 1 = 6 \)
- \( 2! = 2 \times 1 = 2 \)
- \( 1! = 1 \)

Mas e quanto ao \( 0! \)? Para definir o fatorial de zero, \( 0! \), usamos uma convenção matemática. A definição que aceitamos é que:

\[
0! = 1
\]

Mas por que isso?

Vamos entender isso melhor:

1. **Propriedades dos fatoriais**: Uma das propriedades dos fatoriais é q